In [2]:
import os
import boto3
import time
import math
import ctypes
import pandas as pd
import subprocess
import re

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print as dask_print
import distributed
import os
import boto3
import time
import math
import pandas as pd
import subprocess
import re
import concurrent.futures
from osgeo import gdal

# dask/parallelization libraries
import coiled
import dask
from dask.distributed import Client, LocalCluster
from dask.distributed import print
import distributed

# scipy basics
import numpy as np
import rasterio
import rasterio.transform
import rasterio.windows
# import geopandas as gpd
import pandas as pd
import rioxarray
import xarray as xr
from rioxarray.merge import merge_arrays

# numba
from numba import jit
from numba.typed import Dict
from numba.core import types
# scipy basics
import numpy as np
import rasterio
import rasterio.transform
import rasterio.windows
#import geopandas as gpd
import pandas as pd
import rioxarray
import xarray as xr
from rioxarray.merge import merge_arrays

from numba import jit
import concurrent.futures

from osgeo import gdal

### Creating cloud and local clusters

In [1]:
# Full cluster
coiled_cluster = coiled.Cluster(
    n_workers=40,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="10 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "64GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

NameError: name 'coiled' is not defined

In [210]:
# Test cluster
coiled_cluster = coiled.Cluster(
    n_workers=10,
    use_best_zone=True, 
    compute_purchase_option="spot_with_fallback",
    idle_timeout="15 minutes",
    region="us-east-1",
    name="AFOLU_flux_model", 
    #account='jterry64', # Necessary to use the AWS environment that Justin set up in Coiled
    account='wri-forest-research', # Necessary to use the AWS environment that Justin set up in Coiled
    worker_memory = "64GiB" 
)

# Coiled cluster (cloud run)
coiled_client = coiled_cluster.get_client()
coiled_client

2024-05-21 13:12:40,771 - INFO - Using existing cluster: 'AFOLU_flux_model (id: 475693)'
2024-05-21 13:12:40,772 - INFO - Attaching to existing cluster (name: AFOLU_flux_model, https://cloud.coiled.io/clusters/475693?account=wri-forest-research )


Output()

2024-05-21 13:12:41,520 - INFO - Found credentials in shared credentials file: ~/.aws/credentials


<Client: 'tls://10.0.38.7:8786' processes=10 threads=40, memory=147.71 GiB>

In [3]:
# Local single-process cluster (local run). Will run .compute() on just one process, not a whole cluster.
local_client = Client(processes=False)
local_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.200.29:8787/status,
Dashboard: http://172.19.200.29:8787/status,Workers: 1
Total threads: 12,Total memory: 15.46 GiB
Status: running,Using processes: False
Comm: inproc://172.19.200.29/11700/1,Workers: 1
Dashboard: http://172.19.200.29:8787/status,Total threads: 12
Started: Just now,Total memory: 15.46 GiB
Comm: inproc://172.19.200.29/11700/4,Total threads: 12
Dashboard: http://172.19.200.29:41439/status,Memory: 15.46 GiB
Nanny: None,


In [3]:
local_client = Client()
local_client

/home/eglen_wri/anaconda3/envs/dask_notebook/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32881 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:32881/status,
Dashboard: http://127.0.0.1:32881/status,Workers: 4
Total threads: 12,Total memory: 15.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40653,Workers: 4
Dashboard: http://127.0.0.1:32881/status,Total threads: 12
Started: Just now,Total memory: 15.46 GiB
Comm: tcp://127.0.0.1:33321,Total threads: 3
Dashboard: http://127.0.0.1:42473/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:39651,


In [10]:
# Local cluster with multiple workers
local_cluster = LocalCluster()  
local_client = Client(local_cluster)
local_client

/home/eglen_wri/anaconda3/envs/dask_notebook/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41375 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41375/status,
Dashboard: http://127.0.0.1:41375/status,Workers: 4
Total threads: 12,Total memory: 15.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42131,Workers: 4
Dashboard: http://127.0.0.1:41375/status,Total threads: 12
Started: Just now,Total memory: 15.46 GiB
Comm: tcp://127.0.0.1:43109,Total threads: 3
Dashboard: http://127.0.0.1:45207/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:36461,


### Shutting down clusters

In [ ]:
coiled_client.restart() 

In [13]:
coiled_cluster.shutdown()

In [12]:
local_client.shutdown()